In [ ]:
!pip install transformers

In [1]:
# Hack-y way to make relative imports in colab +_+
import sys
sys.path.append('/content/drive/MyDrive/MAIS')

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import torch
import os
import json
from tqdm import tqdm

In [3]:
root_path = "/content/drive/MyDrive/MAIS/"
dataset_path = os.path.join(root_path, "train-balanced-sarcasm.csv")
save_dir = os.path.join(root_path, "data")

In [4]:
df = pd.read_csv(dataset_path)

In [5]:
sarcastic_comments = df[df['label'] == 1].copy()

In [6]:
 sarcastic_comments.reset_index(drop=True, inplace=True)

In [7]:
len(sarcastic_comments[sarcastic_comments['score'] > 30])

15010

In [8]:
sarcastic_comments.drop(labels=['label', 'author', 'subreddit', 'ups', 'downs', 'date', 'created_utc'], axis=1, inplace=True)

In [9]:
len(sarcastic_comments)

505413

In [ ]:
fig = px.pie(sarcastic_comments, 
       names = list(sarcastic_comments.groupby('subreddit')['label'].count().nlargest(20).index), 
       values = sarcastic_comments.groupby('subreddit')['label'].count().nlargest(20)
)
fig.update_traces(textinfo='value')
fig.show()

In [ ]:
max([len(str(x)) for x in sarcastic_comments['comment']])

9921

In [ ]:
list(sarcastic_comments['comment'])[0]

"But they'll have all those reviews!"

In [ ]:
sarcastic_comments['len_comment'] = [len(str(x)) for x in sarcastic_comments['comment']]

In [ ]:
sarcastic_comments['len_parent'] = [len(str(x)) for x in sarcastic_comments['parent_comment']]

In [ ]:
sarcastic_comments.head()

,comment,score,parent_comment,len_comment,len_parent
0,But they'll have all those reviews!,0,"The dumb thing is, they are risking their sell...",35,62
1,wow it is totally unreasonable to assume that ...,2,Clinton campaign accuses FBI of 'blatant doubl...,145,57
2,Ho ho ho... But Melania said that there is no ...,8,Anyone else think that it was interesting the ...,107,135
3,I can't wait until @potus starts a twitter war...,3,Here's what happens when Obama gives up his Tw...,67,59
4,gotta love the teachers who give exams on the ...,3,Monday night Drinking thread Brought to You by...,65,89


In [ ]:
sarcastic_comments[['len_comment', 'len_parent']].describe()

,len_comment,len_parent
count,505413.000000,505413.000000
mean,56.445052,133.164066
std,41.324447,214.586949
min,1.000000,1.000000
25%,30.000000,44.000000
50%,48.000000,77.000000
75%,73.000000,146.000000
max,9921.000000,23256.000000


In [ ]:
px.box(sarcastic_comments[['len_comment', 'len_parent']], y='len_comment')

In [ ]:
px.box(sarcastic_comments[['len_comment', 'len_parent']], y='len_parent')

In [ ]:
# Remove parent comment outliers having length more than 1000
sarcastic_comments = sarcastic_comments[sarcastic_comments['len_parent'] < 1000]

In [ ]:
# Remove comment outliers (very long replies)
sarcastic_comments = sarcastic_comments[sarcastic_comments['len_comment'] < 1000]

In [ ]:
# !rm /content/drive/MyDrive/MAIS/data/*

In [ ]:
len(list(os.listdir('/content/drive/MyDrive/MAIS/data')))

501098

In [78]:
test = list(sarcastic_comments.iloc[66666, :])
print("Parent comment: ", test[-1])
print()
print("Reply: ", test[0])

Parent comment:  Why isn't golem + rage as good as it sounds?

Reply:  because then it'd be Golage


,comment,score,parent_comment
0,But they'll have all those reviews!,0,"The dumb thing is, they are risking their sell..."
1,wow it is totally unreasonable to assume that ...,2,Clinton campaign accuses FBI of 'blatant doubl...
2,Ho ho ho... But Melania said that there is no ...,8,Anyone else think that it was interesting the ...
3,I can't wait until @potus starts a twitter war...,3,Here's what happens when Obama gives up his Tw...
4,gotta love the teachers who give exams on the ...,3,Monday night Drinking thread Brought to You by...
